# Cài đặt thư viện

In [155]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier # CART
from sklearn.metrics import accuracy_score, classification_report, mean_squared_error
from sklearn.model_selection import GridSearchCV

# Tiền xử lý dữ liệu

In [156]:
df = pd.read_csv('./fetal_health.csv')

# display(df.head(5))

df.isnull().sum()

baseline value                                            0
accelerations                                             0
fetal_movement                                            0
uterine_contractions                                      0
light_decelerations                                       0
severe_decelerations                                      0
prolongued_decelerations                                  0
abnormal_short_term_variability                           0
mean_value_of_short_term_variability                      0
percentage_of_time_with_abnormal_long_term_variability    0
mean_value_of_long_term_variability                       0
histogram_width                                           0
histogram_min                                             0
histogram_max                                             0
histogram_number_of_peaks                                 0
histogram_number_of_zeroes                                0
histogram_mode                          

# Model Decision Tree (CART)

Trước khi tối ưu hóa

In [157]:
X = df.drop(columns=['fetal_health'])
y = df['fetal_health']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# mô hình Decision Tree
model = DecisionTreeClassifier(criterion='gini', random_state=0)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

# Đánh giá
print('Classification Report:')
print(classification_report(y_test, y_pred))
accuracy = accuracy_score(y_test, y_pred)
print('Độ chính xác của mô hình là:', accuracy)

Classification Report:
              precision    recall  f1-score   support

         1.0       0.97      0.94      0.96       333
         2.0       0.76      0.86      0.81        64
         3.0       0.88      0.97      0.92        29

    accuracy                           0.93       426
   macro avg       0.87      0.92      0.89       426
weighted avg       0.93      0.93      0.93       426

Độ chính xác của mô hình là: 0.9295774647887324


Sau khi tối ưu hóa (sử dụng tinh chỉnh siêu tham số sử dụng Grid Search)

In [158]:
# Danh sách các giá trị max_depth cần tinh chỉnh
param_grid = {
    'max_depth': [5, 10, 15, 20, 25, 30],  
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

X = df.drop(columns=['fetal_health'])
y = df['fetal_health']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# mô hình Decision Tree
model = DecisionTreeClassifier(criterion='gini', random_state=0)

# Tìm kiếm siêu tham số tốt nhất bằng Grid Search và kiểm tra chéo 5-fold
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# In ra giá trị tốt nhất trên tập kiểm tra
print("Best max_depth:", grid_search.best_params_['max_depth'])
print("Best min_samples_split:", grid_search.best_params_['min_samples_split'])
print("Best min_samples_leaf:", grid_search.best_params_['min_samples_leaf'])

best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
print('Classification Report:')
print(classification_report(y_test, y_pred))
print('Độ chính xác của mô hình là:', accuracy_score(y_test, y_pred))


Best max_depth: 5
Best min_samples_split: 2
Best min_samples_leaf: 4
Classification Report:
              precision    recall  f1-score   support

         1.0       0.95      0.97      0.96       333
         2.0       0.88      0.70      0.78        64
         3.0       0.83      1.00      0.91        29

    accuracy                           0.93       426
   macro avg       0.89      0.89      0.88       426
weighted avg       0.93      0.93      0.93       426

Độ chính xác của mô hình là: 0.931924882629108
